# 时间序列处理
* 用resampling改变时间频率
* 缺失值填充
* 数据对齐
* Rolling
* Expanding
#### 处理dataframe时，经常会遇到时间类型数据，本次课会介绍时间序列处理的常见function

In [1]:
import pandas as pd
import numpy as np


## 用resampling改变时间频率

In [2]:
df=pd.read_csv('./data/stock.csv',parse_dates=['date'])

In [4]:
df

,close,date,ticker
0,776.60,2015-12-29,GOOG
1,771.00,2015-12-30,GOOG
2,758.88,2015-12-31,GOOG
3,741.84,2016-01-04,GOOG
4,108.74,2015-12-29,AAPL
5,107.32,2015-12-30,AAPL
6,105.26,2015-12-31,AAPL
7,105.35,2016-01-04,AAPL


In [4]:
grp=df.set_index('date').resample('MS')

#### 和groupby一样

In [5]:
grp['close'].mean()

date
2015-12-01    437.966667
2016-01-01    423.595000
Freq: MS, Name: close, dtype: float64

In [6]:
grp['close'].agg(['mean','sum'])

,mean,sum
date,,
2015-12-01,437.966667,2627.80
2016-01-01,423.595000,847.19


In [7]:
df['month']=df['date'].dt.month
df['year']=df['date'].dt.year

In [8]:
grp=df.groupby(['year','month'])

In [9]:
grp['close'].mean()

year  month
2015  12       437.966667
2016  1        423.595000
Name: close, dtype: float64

In [10]:
grp['close'].agg(['mean','sum'])

,,mean,sum
year,month,,
2015,12,437.966667,2627.80
2016,1,423.595000,847.19


## 缺失值填充

In [11]:
df

,close,date,ticker,month,year
0,776.60,2015-12-29,GOOG,12,2015
1,771.00,2015-12-30,GOOG,12,2015
2,758.88,2015-12-31,GOOG,12,2015
3,741.84,2016-01-04,GOOG,1,2016
4,108.74,2015-12-29,AAPL,12,2015
5,107.32,2015-12-30,AAPL,12,2015
6,105.26,2015-12-31,AAPL,12,2015
7,105.35,2016-01-04,AAPL,1,2016


In [12]:
df.loc[[0,2,3],'close']=np.nan
df

,close,date,ticker,month,year
0,NaN,2015-12-29,GOOG,12,2015
1,771.00,2015-12-30,GOOG,12,2015
2,NaN,2015-12-31,GOOG,12,2015
3,NaN,2016-01-04,GOOG,1,2016
4,108.74,2015-12-29,AAPL,12,2015
5,107.32,2015-12-30,AAPL,12,2015
6,105.26,2015-12-31,AAPL,12,2015
7,105.35,2016-01-04,AAPL,1,2016


In [13]:
df.set_index('date',inplace=True)


In [14]:
df

,close,ticker,month,year
date,,,,
2015-12-29,NaN,GOOG,12,2015
2015-12-30,771.00,GOOG,12,2015
2015-12-31,NaN,GOOG,12,2015
2016-01-04,NaN,GOOG,1,2016
2015-12-29,108.74,AAPL,12,2015
2015-12-30,107.32,AAPL,12,2015
2015-12-31,105.26,AAPL,12,2015
2016-01-04,105.35,AAPL,1,2016


In [15]:
df['close'].fillna(method='ffill')

date
2015-12-29       NaN
2015-12-30    771.00
2015-12-31    771.00
2016-01-04    771.00
2015-12-29    108.74
2015-12-30    107.32
2015-12-31    105.26
2016-01-04    105.35
Name: close, dtype: float64

In [16]:
df['close'].fillna(method='bfill')

date
2015-12-29    771.00
2015-12-30    771.00
2015-12-31    108.74
2016-01-04    108.74
2015-12-29    108.74
2015-12-30    107.32
2015-12-31    105.26
2016-01-04    105.35
Name: close, dtype: float64

In [17]:
df['close'].ffill()

date
2015-12-29       NaN
2015-12-30    771.00
2015-12-31    771.00
2016-01-04    771.00
2015-12-29    108.74
2015-12-30    107.32
2015-12-31    105.26
2016-01-04    105.35
Name: close, dtype: float64

In [18]:
df['close'].bfill()

date
2015-12-29    771.00
2015-12-30    771.00
2015-12-31    108.74
2016-01-04    108.74
2015-12-29    108.74
2015-12-30    107.32
2015-12-31    105.26
2016-01-04    105.35
Name: close, dtype: float64

## 数据对齐

In [19]:
df

,close,ticker,month,year
date,,,,
2015-12-29,NaN,GOOG,12,2015
2015-12-30,771.00,GOOG,12,2015
2015-12-31,NaN,GOOG,12,2015
2016-01-04,NaN,GOOG,1,2016
2015-12-29,108.74,AAPL,12,2015
2015-12-30,107.32,AAPL,12,2015
2015-12-31,105.26,AAPL,12,2015
2016-01-04,105.35,AAPL,1,2016


In [20]:
ts=pd.date_range('2015-12-30',periods=10,freq='B')
ts

DatetimeIndex(['2015-12-30', '2015-12-31', '2016-01-01', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-11', '2016-01-12'],
              dtype='datetime64[ns]', freq='B')

In [21]:
df=df.reset_index().drop_duplicates(subset=['date'],keep='last').set_index('date')
df

,close,ticker,month,year
date,,,,
2015-12-29,108.74,AAPL,12,2015
2015-12-30,107.32,AAPL,12,2015
2015-12-31,105.26,AAPL,12,2015
2016-01-04,105.35,AAPL,1,2016


In [22]:
df.reindex(ts)

,close,ticker,month,year
2015-12-30,107.32,AAPL,12.0,2015.0
2015-12-31,105.26,AAPL,12.0,2015.0
2016-01-01,NaN,NaN,NaN,NaN
2016-01-04,105.35,AAPL,1.0,2016.0
2016-01-05,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN
2016-01-11,NaN,NaN,NaN,NaN
2016-01-12,NaN,NaN,NaN,NaN


### Rolling 

In [23]:
df

,close,ticker,month,year
date,,,,
2015-12-29,108.74,AAPL,12,2015
2015-12-30,107.32,AAPL,12,2015
2015-12-31,105.26,AAPL,12,2015
2016-01-04,105.35,AAPL,1,2016


In [24]:
df['close'].rolling(2).mean()

date
2015-12-29        NaN
2015-12-30    108.030
2015-12-31    106.290
2016-01-04    105.305
Name: close, dtype: float64

In [25]:
df['close'].rolling(2,min_periods=1).mean()

date
2015-12-29    108.740
2015-12-30    108.030
2015-12-31    106.290
2016-01-04    105.305
Name: close, dtype: float64

In [26]:
df['close'].rolling(3).mean()

date
2015-12-29           NaN
2015-12-30           NaN
2015-12-31    107.106667
2016-01-04    105.976667
Name: close, dtype: float64

In [27]:
df['close'].rolling(3,center=True).mean()

date
2015-12-29           NaN
2015-12-30    107.106667
2015-12-31    105.976667
2016-01-04           NaN
Name: close, dtype: float64

In [28]:
df['close'].rolling(3,min_periods=1).mean()

date
2015-12-29    108.740000
2015-12-30    108.030000
2015-12-31    107.106667
2016-01-04    105.976667
Name: close, dtype: float64

In [29]:
df

,close,ticker,month,year
date,,,,
2015-12-29,108.74,AAPL,12,2015
2015-12-30,107.32,AAPL,12,2015
2015-12-31,105.26,AAPL,12,2015
2016-01-04,105.35,AAPL,1,2016


In [30]:
df['close'].rolling(3,min_periods=1,center=True).mean()

date
2015-12-29    108.030000
2015-12-30    107.106667
2015-12-31    105.976667
2016-01-04    105.305000
Name: close, dtype: float64

### Expanding

In [42]:
df['close'].expanding(min_periods=1).sum()

date
2015-12-29    108.74
2015-12-30    216.06
2015-12-31    321.32
2016-01-04    426.67
Name: close, dtype: float64

In [43]:
df['close'].expanding(2).sum()

date
2015-12-29       NaN
2015-12-30    216.06
2015-12-31    321.32
2016-01-04    426.67
Name: close, dtype: float64

In [44]:
df['close'].expanding(3).sum()

date
2015-12-29       NaN
2015-12-30       NaN
2015-12-31    321.32
2016-01-04    426.67
Name: close, dtype: float64

### 课后练习

In [97]:
sp5_df = pd.read_csv(
    'sp500.csv', usecols=['date', 'adj_close'], 
    parse_dates=['date'])

计算每一天adj_close相较于前一天变化的百分比

用resample计算sp5_df的 adj_close 的月平均值